# Profiling MXNet Models

In this tutorial, we will learn how to profile MXNet models to measure their running time and memory consumption using the MXNet profiler.

## The incorrect way to profile

If you have just begun using MXNet, you might be tempted to measure execution time of your code using Python's `time` module like shown below:

In [1]:
from time import time
from mxnet import autograd, nd
import mxnet as mx

start = time()

x = nd.random_uniform(shape=(2000,2000))
y = nd.dot(x, x)

print('Time for matrix multiplication: %f sec\n' % (time() - start))
                                
print(y.asnumpy())
                                
print('Time for printing the output: %f sec' % (time() - start))

/home/ubuntu/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Time for matrix multiplication: 0.001189 sec

[[501.1584  508.29724 495.65237 ... 492.84705 492.69092 490.0481 ]
 [508.81058 507.1822  495.1743  ... 503.10526 497.29315 493.67917]
 [489.56598 499.47015 490.17722 ... 490.99945 488.05008 483.28836]
 ...
 [484.0019  495.7179  479.92142 ... 493.69952 478.89194 487.2074 ]
 [499.64932 507.65094 497.5938  ... 493.0474  500.74512 495.82712]
 [516.0143  519.1715  506.354   ... 510.08878 496.35608 495.42523]]
Time for printing the output: 0.170921 sec


Like you can see, it seems as if printing the output takes lot more time that multiplying two large matrices. That doesn't feel right. 

This is because in MXNet, all operations are executed asynchronously. So, when `nd.dot(x, x)` returns, the matrix multiplication is not complete, it has only been queued for execution. `asnumpy` in `print(y.asnumpy())` however waits for the result to be computed and hence takes longer time.

## The correct way to profile

The correct way to measure running time of MXNet models is to use MXNet profiler. In the rest of this tutorial, we will learn how to use the MXNet profiler to measure the running time and memory consumption of MXNet models.

Let's begin by importing the profiler and configuring it.

In [2]:
from mxnet import profiler
profiler.set_config(profile_all=True, aggregate_stats=True, filename='profile_output.json')

`profile_all` enables all types of profiling. You can also individually enable the following types of profiling:

- `profile_symbolic`: whether to profile symbolic operators
- `profile_imperative`: whether to profile imperative operators
- `profile_memory`: whether to profile memory usage
- `profile_api`: whether to profile the C API

`aggregate_stats` aggregates statistics in memory which can then be printed to console by calling `profiler.dumps()`.

### Setup: Build a model

Let's build a small model (LeNet) that we can use for profiling.

In [3]:
from mxnet import gluon
net = gluon.nn.HybridSequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(512, activation="relu"))
    net.add(gluon.nn.Dense(10))

We need data that we can run through the network for profiling. We'll use MNIST dataset.

In [4]:
from mxnet.gluon.data.vision import transforms
train_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=True).transform_first(transforms.ToTensor()),
                                   batch_size=64, shuffle=True)

Let's define a method that will run one training iteration given data and label.

In [5]:
# Use GPU is available
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()

# Initialize the parameters with random weights
net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

# Use SGD optimizer
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

# Softmax Cross Entropy is a frequently used loss function for multi-classs classification
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

# A helper function to run one training iteration
def run_training_iteration(data, label):
    
    # Load data and label is the right context
    data = data.as_in_context(ctx)
    label = label.as_in_context(ctx)
    
    # Run the forward pass
    with autograd.record():
        output = net(data)
        loss = softmax_cross_entropy(output, label)
    
    # Run the backward pass
    loss.backward()
    
    # Apply changes to parameters
    trainer.step(data.shape[0])

### Starting and stopping the profiler

When the first forward pass is run on a network, MXNet does a number of housekeeping tasks including infering the shapes of various parameters and allocating memory for intermediate and final outputs, etc. For these reasons, profiling the first iteration doesn't provide accurate results. We will therefore skip the first iteration.

In [6]:
# Run the first iteration without profiling
itr = iter(train_data)
run_training_iteration(*next(itr))

We'll run the next iteration with the profiler turned on.

In [7]:

data, label = next(itr)

# Ask the profiler to start recording
profiler.set_state('run')

run_training_iteration(*next(itr))

# Ask the profiler to stop recording
profiler.set_state('stop')

### Viewing profiler output

There are two ways to view the information collected by the profiler. You can either view it in console or you can view a more graphical version in a browser.

#### 1. View in console

You can use the `profiler.dumps()` method to view the information collected by the profiler in the console. The collected information contains time taken by each operator, time taken by each C API and memory consumed in both CPU and GPU.

In [8]:
print(profiler.dumps())


Profile Statistics.
	Note that counter items are counter values and not time units.
Device Storage
Name                          Total Count        Time (ms)    Min Time (ms)    Max Time (ms)    Avg Time (ms)
----                          -----------        ---------    -------------    -------------    -------------
Memory: cpu/0                         132           0.0000           0.0000         401.6640         200.8320
Memory: gpu/0                          43        5320.7041         200.7040        6099.2002        2949.2480

MXNET_C_API
Name                          Total Count        Time (ms)    Min Time (ms)    Max Time (ms)    Avg Time (ms)
----                          -----------        ---------    -------------    -------------    -------------
MXNDArrayReshape64                      1           0.0610           0.0610           0.0610           0.0610
MXAutogradBackwardEx                    1           0.7250           0.7250           0.7250           0.7250
MXAutog

#### 2. View in browser

You can also dump the information collected by the profiler into a `json` file using the `profiler.dump()` function and view it in a browser.

In [9]:
profiler.dump()

`dump()` creates a `json` file which can be viewed using a trace consumer like chrome://tracing. Here is a snapshot that shows a trace output.

![Tracing Screenshot](profiler_output_chrome.png)

Let's zoom in to check the time taken by operators

![Operator profiling](profile_operators.png)